# Cleaning Data

**Prerequisites**

- [Intro](intro.ipynb)  
- [Boolean selection](basics.ipynb)  
- [Indexing](the_index.ipynb)  


**Outcomes**

- Be able to use string methods to clean data that comes in as a string  
- Be able to drop missing data  
- Use cleaning methods to prepare and analyze a real dataset  


**Data**

- Item information from about 3,000 Chipotle meals from about 1,800
  Grubhub orders  

In [ ]:
# Uncomment following line to install on colab
#! pip install qeds

In [1]:
import pandas as pd
import numpy as np
#import qeds

## Outline

- [Cleaning Data](#Cleaning-Data)  
  - [Cleaning data](#Cleaning-data)  
  - [String methods](#String-methods)  
  - [Type conversions](#Type-conversions)  
  - [Missing data](#Missing-data)  
  - [Case Study](#Case-Study)  
  - [Appendix: performance of `.str` methods](#Appendix:-performance-of-`.str`-methods)  
  - [Exercises](#Exercises)  

## Cleaning data

For many data projects, a [significant proportion of
time](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#74d447456f63)
is spent collecting and cleaning the data — not doing the analysis

This non-analysis work is often called “data cleaning”

Pandas provides very powerful tools for doing data cleaning, which we
will demonstrate using the following dataset

In [2]:
# np.nan can generate an NA 

df = pd.DataFrame({"numbers": ["#23", "#24", "#18", "#14", "#12", "#10", "#35"],
                   "nums": ["23", "24", "18", "14", np.nan, "XYZ", "35"],
                   "colors": ["green", "red", "yellow", "orange", "purple", "blue", "pink"],
                   "other_column": [0, 1, 0, 2, 1, 0, 2]})
df

,numbers,nums,colors,other_column
0,#23,23,green,0
1,#24,24,red,1
2,#18,18,yellow,0
3,#14,14,orange,2
4,#12,NaN,purple,1
5,#10,XYZ,blue,0
6,#35,35,pink,2


In [3]:
df["numbers"].mean()

# report errors because the type of "nums" column is string rather than float or int

TypeError: Could not convert #23#24#18#14#12#10#35 to numeric

What would happen if we wanted to try and compute the mean of
`numbers`?

```python
df["numbers"].mean()
```


It throws an error!

Can you figure out why?

Hint: When looking at error messages, it helps to start at the very
bottom

The final error says, `TypeError: Could not convert #23#24... to numeric`


<a id='exercise-0'></a>
> See exercise 1 in the [*exercise list*](#exerciselist-0)

## String methods

Our solution to the previous exercise was to remove the `#` by using
the `replace` string method: `int(c2n.replace("#", ""))`

One way to make this change to every element of a column would be to
loop through all elements of the column and apply the desired string
methods…

In [4]:
%%time

# Iterate over all rows
for row in df.iterrows():

    # `iterrows` method produces a tuple with two elements...
    # The first element is an index and second is a series with the data from that row
    index_value, column_values = row

    # Apply string method
    clean_number = int(column_values["numbers"].replace("#", ""))

    # The `at` method is very similar to the `loc` method, but it is specialized
    # for accessing single elements at a time... We wanted to use it here to give
    # the loop the best chance to beat a faster method we show you next
    
    # "numbers_loop" here creats a new column that stores the updated numbers in floating type
    df.at[index_value, "numbers_loop"] = clean_number
    


Wall time: 4.99 ms


While this is fast for a small dataset like this, it turns out to be
slow for larger datasets

There is a *significantly* faster (and easier) way to apply a string
method to an entire column of data

Most of the methods that are available to a Python string (we learned a
few of them in the [strings lecture](python_fundamentals/basics.ipynb)) are
available to a pandas Series that has `dtype` object

We access them by doing `s.str.method_name` where `method_name` is
the name of the method

When we apply the method to a Series, it is applied to all rows in the
series in one shot!

Let’s redo our previous example using a Pandas `.str` method

In [5]:
%%time

# ~2x faster than loop... However speed gain increases with size of DataFrame. The
# speedup can be in the ballpark of ~100-500x faster for big DataFrames
# See appendix at the end of the lecture for an application on a larger DataFrame
df["numbers_str"] = df["numbers"].str.replace("#", "")

df["numbers_str"]

Wall time: 1.96 ms


0    23
1    24
2    18
3    14
4    12
5    10
6    35
Name: numbers_str, dtype: object

We can use `.str` to access almost any string method that works on
normal strings (see the [official
documentation](https://pandas.pydata.org/pandas-docs/stable/text.html)
for more information)

In [6]:
df["colors"].str.contains("p")

0    False
1    False
2    False
3    False
4     True
5    False
6     True
Name: colors, dtype: bool

In [7]:
df["colors"].str.capitalize()

0     Green
1       Red
2    Yellow
3    Orange
4    Purple
5      Blue
6      Pink
Name: colors, dtype: object

In [8]:
# Exercise 2

# Make a new column called colors_upper that contains the elements of colors with all uppercase letters.

df["colors_upper"] = df["colors"].str.upper()
df["colors_upper"]

0     GREEN
1       RED
2    YELLOW
3    ORANGE
4    PURPLE
5      BLUE
6      PINK
Name: colors_upper, dtype: object


<a id='exercise-1'></a>
> See exercise 2 in the [*exercise list*](#exerciselist-0)

## Type conversions

In our example above, even after we have removed the `"#"`, the
`dtype` of the `numbers_str` column shows that pandas still treats
it as a string

We need to convert this column to numbers

The best way to do this is using the `pd.to_numeric` function

This method attempts to convert whatever is stored in a Series into
numeric values

For example, after having removed the `"#"` from the numbers in column
`"numbers"`, they are ready to be converted to numbers

In [9]:
df["numbers_numeric"] = pd.to_numeric(df["numbers_str"])

In [10]:
df.dtypes

numbers             object
nums                object
colors              object
other_column         int64
numbers_loop       float64
numbers_str         object
colors_upper        object
numbers_numeric      int64
dtype: object

In [11]:
df.head()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
4,#12,NaN,purple,1,12.0,12,PURPLE,12


There is support for converting to other types well

Using the `astype` method we can convert to any of the supported
pandas `dtypes` (recall the [intro lecture](intro.ipynb))

Below are some examples (pay attention to the reported `dtype`)

In [12]:
df["numbers_numeric"].astype(str)

0    23
1    24
2    18
3    14
4    12
5    10
6    35
Name: numbers_numeric, dtype: object

In [13]:
df["numbers_numeric"].astype(float)

0    23.0
1    24.0
2    18.0
3    14.0
4    12.0
5    10.0
6    35.0
Name: numbers_numeric, dtype: float64


<a id='exercise-2'></a>
> See exercise 3 in the [*exercise list*](#exerciselist-0)

In [14]:
# Exercise 3

# Convert the column "nums" to a numeric type using pd.to_numeric and save it to the DataFrame as "nums_tonumeric"

# Notice that there is a missing value and a value that is not a number

pd.to_numeric(df["nums"], errors = "ignore")
# errors = "ignore" means keep the original values of the elements that can not be converted to numeric values

pd.to_numeric(df["nums"], errors = "coerce")
# errors = "coerce" means to convert all the values to NaN for the elements that can not be converted to numeric values

# Look at the documentation for pd.to_numeric and think about how to overcome this

# Think about why this could be a bad idea of used without knowing what your data looks like 
# (Think about what happens when you apply it to the "numbers" column before replacing the "#")



0    23.0
1    24.0
2    18.0
3    14.0
4     NaN
5     NaN
6    35.0
Name: nums, dtype: float64

## Missing data

Many datasets have missing data

In our example, we are missing an element from the `"nums"` column

In [15]:
df

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
4,#12,NaN,purple,1,12.0,12,PURPLE,12
5,#10,XYZ,blue,0,10.0,10,BLUE,10
6,#35,35,pink,2,35.0,35,PINK,35


We can find missing data by using the `isnull` method

In [16]:
df.isnull()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False


We might want to know whether particular rows or columns have any
missing data

To do this we can use the `.any` method on the boolean DataFrame
`df.isnull()`

In [19]:
df.isnull().any(axis=0)

numbers            False
nums                True
colors             False
other_column       False
numbers_loop       False
numbers_str        False
colors_upper       False
numbers_numeric    False
dtype: bool

In [20]:
df.isnull().any(axis=1)

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

There are many approaches to dealing with missing data

Two that are commonly used (and the corresponding DataFrame method) are

- Exclusion: Ignore any data that is missing (`.dropna`)  
- Imputation: Compute “predicted” values for the data that is missing
  (`.fillna`)  


For the advantages and disadvantages of these (and other) approaches,
consider reading the [Wikipedia
article](https://en.wikipedia.org/wiki/Missing_data)

For now, let’s see some examples

In [21]:
# drop all rows containing a missing observation
df.dropna()

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
5,#10,XYZ,blue,0,10.0,10,BLUE,10
6,#35,35,pink,2,35.0,35,PINK,35


In [22]:
# fill the missing values with a specific value
df.fillna(value=100)

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
4,#12,100,purple,1,12.0,12,PURPLE,12
5,#10,XYZ,blue,0,10.0,10,BLUE,10
6,#35,35,pink,2,35.0,35,PINK,35


In [23]:
# use the _next_ valid observation to fill the missing data
df.fillna(method="bfill")

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
4,#12,XYZ,purple,1,12.0,12,PURPLE,12
5,#10,XYZ,blue,0,10.0,10,BLUE,10
6,#35,35,pink,2,35.0,35,PINK,35


In [24]:
# use the _previous_ valid observation to fill missing data
df.fillna(method="ffill")

,numbers,nums,colors,other_column,numbers_loop,numbers_str,colors_upper,numbers_numeric
0,#23,23,green,0,23.0,23,GREEN,23
1,#24,24,red,1,24.0,24,RED,24
2,#18,18,yellow,0,18.0,18,YELLOW,18
3,#14,14,orange,2,14.0,14,ORANGE,14
4,#12,14,purple,1,12.0,12,PURPLE,12
5,#10,XYZ,blue,0,10.0,10,BLUE,10
6,#35,35,pink,2,35.0,35,PINK,35


We will see more examples of dealing with missing data in future
chapters.

## Case Study

We will now use data from an
[article](https://www.nytimes.com/interactive/2015/02/17/upshot/what-do-people-actually-order-at-chipotle.html)
written by The Upshot at the NYTimes

This data has order information from almost 2,000 Chipotle orders and
includes information on what was ordered and how much it cost

In [49]:
#import qeds
#chipotle = qeds.data.load("chipotle_raw")
#chipotle.head()




<a id='exercise-3'></a>
> See exercise 4 in the [*exercise list*](#exerciselist-0)

## Appendix: performance of `.str` methods

Let’s repeat the “remove the `#`” example from above, but this time on
a much larger dataset

In [28]:
import numpy as np
test = pd.DataFrame({"floats": np.round(100*np.random.rand(100000), 2)})
test["strings"] = test["floats"].astype(str) + "%"
test.head()

,floats,strings
0,12.38,12.38%
1,48.77,48.77%
2,82.09,82.09%
3,45.46,45.46%
4,8.97,8.97%


In [31]:
%%time

for row in test.iterrows():
    index_value, column_values = row
    clean_number = column_values["strings"].replace("%", "")
    test.at[index_value, "numbers_loop"] = clean_number

Wall time: 9.13 s


In [36]:
%%time
test["numbers_str_method"] = test["strings"].str.replace("%", "")

test.head()

Wall time: 74.8 ms


,floats,strings,numbers_loop,numbers_str_method
0,12.38,12.38%,12.38,12.38
1,48.77,48.77%,48.77,48.77
2,82.09,82.09%,82.09,82.09
3,45.46,45.46%,45.46,45.46
4,8.97,8.97%,8.97,8.97


In [48]:
%%time

# this is to test if the two columns are the same (the result is true only when every element in the first column
# matches the one in the second)
test["numbers_str_method"].equals(test["numbers_str_method"])

Wall time: 16 ms


True

In [46]:
%%time

# this is to see if items in these two columns match one by one
test["numbers_str_method"] == (test["numbers_loop"])

Wall time: 9.97 ms


0        True
1        True
2        True
3        True
4        True
         ... 
99995    True
99996    True
99997    True
99998    True
99999    True
Length: 100000, dtype: bool

We got the exact same result in a fraction of the time!

## Exercises


<a id='exerciselist-0'></a>
**Exercise 1**

Convert the string below into a number

In [8]:
c2n = "#39"

c2n = int(c2n.replace("#", ""))

type(c2n)

int

([*back to text*](#exercise-0))

**Exercise 2**

Make a new column called `colors_upper` that contains the elements of
`colors` with all uppercase letters.

([*back to text*](#exercise-1))

**Exercise 3**

Convert the column `"nums"` to a numeric type using `pd.to_numeric` and
save it to the DataFrame as `"nums_tonumeric"`

Notice that there is a missing value and a value that is not a number

Look at the documentation for `pd.to_numeric` and think about how to
overcome this

Think about why this could be a bad idea of used without
knowing what your data looks like (Think about what happens when you
apply it to the `"numbers"` column before replacing the `"#"`)

([*back to text*](#exercise-2))

**Exercise 4**

We'd like you to use this data to answer the following questions

- What is the average price of an item with chicken?  
- What is the average price of an item with steak?  
- Did chicken or steak produce more revenue (total)?  
- How many missing items are there in this dataset? How many missing
  items in each column?  


Hint: before you will be able to do any of these things you will need to
make sure the `item_price` column has a numeric `dtype` (probably
float)

([*back to text*](#exercise-3))